## CNN: EfficientNetB2
![EfficientNet-B2](https://viso.ai/wp-content/uploads/2024/03/EfficientNet-Architecture-diagram.png)
* Fuente: https://viso.ai/deep-learning/efficientnet/

### IMPORTAMOS LIBRERIAS

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
import numpy as np
import sys
import datetime
from datetime import datetime

### CONFIGURACIÓN GENERAL (HIPERPARAMETROS) util dado la
interpolacion de las variables constantes, simplifica el input pero puedes perder fine-grained style information

*   Tamaño de batch: necesario encontrar un balance entre el costo computacional y la estabilidad de gradientes
*   Resolución por imágenes (256x256) [Art-Bench-10](https://github.com/liaopeiyuan/artbench)
*   Cantidad de epocas por fase 1, 2 y 3.


In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256) # 256x256 por pruebas
EPOCHS_PHASE1 = 5
EPOCHS_PHASE2 = 5
EPOCHS_PHASE3 = 5
EMBEDDING_DIM = 256

### DATASET LOADING
Se cargan datasets de entrenamiento y prueba desde directorios, usando etiquetas enteras.

> Importante, prefetch() y AUTOTUNE: realiza en paralelo el procesamiento de data (test/train), además del AUTOTUNE, ayuda a obtener el número óptimo de batches a preprocesar en paralelo



In [ ]:
def load_datasets():
    train_ds = image_dataset_from_directory(
        "C:/workspace/data/train_sampled", # 5000 imagenes
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode='int',
        shuffle=True
    )

    test_ds = image_dataset_from_directory(
        "C:/workspace/data/test",
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode='int',
        shuffle=False
    )

    return train_ds.prefetch(tf.data.AUTOTUNE), test_ds.prefetch(tf.data.AUTOTUNE)

### AUGMENTACIÓN
Al aumentar la data, debido a la falta de recursos para el entrenamineto lo que realizamos es generar variaciones de manera distintas a partir de las ingresadas, lo que mejora la generalización del modelo frente a variaciones pequeñas de estilo, manteniendo coherencia semántica.

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),  # simula reflejo horizontal
    layers.RandomRotation(0.05),  # aplica rotaciones pequeñas, evitando la distorsion de estilo
    layers.RandomZoom(height_factor=0.1, width_factor=0.1),  # escalar levemente
    layers.Lambda(preprocess_input)  # importante: normalizar valores, asi comprende mejor el modelo
], name="data_augmentation")

### MODELO BASE
El modelo se construye sin la cabeza superior, la cual es la que define el final o el enfoque de clasificación de imagenes con una. Si trainable_layers=None, se congela toda la red.

In [ ]:
def build_model(trainable_layers=None):
    base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE, 3))
    base_model.trainable = trainable_layers is not None

    if trainable_layers:
        for layer in base_model.layers[:-trainable_layers]:
            layer.trainable = False  # solo entrenamos la parte final del backbone (forma de referenciar la arquitectura o cuerpo interno de la red)

    inputs = layers.Input(shape=(*IMG_SIZE, 3))
    x = data_augmentation(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x) # reduccion del mapa de caracteristicas
    x = layers.BatchNormalization()(x) # durante el entrenamiento impacta más con su estabilizacion
    x = layers.Dropout(0.3)(x)
    embeddings = layers.Dense(EMBEDDING_DIM, activation=None, name="embedding")(x)
    outputs = layers.Dense(10, activation='softmax')(embeddings) # aplicando su clasificacion sobre 10 clases (estilos dado el dataset)

    model = models.Model(inputs, outputs)
    model.summary()
    return model

### CALLBACKS

*   early stopping
*   reducción de LR
*   evaluación de embeddings por época

In [ ]:
def get_callbacks(phase, model=None, dataset=None):
    base_callbacks =  [
        callbacks.EarlyStopping(patience=3, restore_best_weights=True), # detiene el proceso de treno si presencia una NO mejoria
        callbacks.ModelCheckpoint(f"efficientnet_phase{phase}.keras", save_best_only=True), # produce el binario por fase, para retomar sin reinicar todo
        callbacks.ReduceLROnPlateau(factor=0.5, patience=2) # reducimos la tasa de aprendizaje en caso no mejores, aplica esto luego de 2 epocas sin mejora
    ]

    if model and dataset:
        eval_callback = callbacks.LambdaCallback(
            on_epoch_end=lambda epoch, logs: evaluate_embeddings(model, dataset, silent=True)
        )
        base_callbacks.append(eval_callback)

    return base_callbacks

### SCHEDULE DE LEARNING RATE TRIANGULAR
Reduce gradualmente la tasa de aprendizaje para permitir rápido ascenso y luego descenso gradual, esto se da por fase (QUIZÁ UN POCO REDUNDANTE)

In [ ]:
def slanted_triangular_lr(max_lr, total_epochs):
    def scheduler(epoch):
        pct = epoch / total_epochs
        if pct < 0.3:
            return max_lr * (pct / 0.3)
        else:
            return max_lr * (1 - (pct - 0.3) / 0.7)
    return scheduler

### ENTRENAMIENTO

In [ ]:
def train_model():
    train_ds, test_ds = load_datasets()

    # FASE 1: Feature Extraction (RED CONGELADA)
    model = build_model(trainable_layers=None)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), # tasa de aprendizaje probablemente muy alta para ser las capas primarias (generales) - la base
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS_PHASE1, callbacks=get_callbacks(1, model, test_ds), verbose=1)

    # FASE 2: Gradual Unfreezing desde block6 (estimado: x últimas capas)
    model = build_model(trainable_layers=25)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    lr_scheduler = callbacks.LearningRateScheduler(slanted_triangular_lr(1e-4, EPOCHS_PHASE2))
    model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS_PHASE2, callbacks=[lr_scheduler] + get_callbacks(2, model, test_ds), verbose=1)

    # FASE 3: Full Fine-Tuning
    model = build_model(trainable_layers=len(EfficientNetB2(weights='imagenet', include_top=False).layers))
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS_PHASE3, callbacks=get_callbacks(3, model, test_ds), verbose=1)

    return model, test_ds

### EVALUACIÓN EMBEDDINGS
Se extraen los embeddings desde la capa 'embedding' y se evalúan con silhouette y P@5.

In [ ]:
def evaluate_embeddings(model, dataset, silent=False):
    try:
        feature_model = models.Model(inputs=model.input, outputs=model.get_layer("embedding").output)
        embeddings = feature_model.predict(dataset, verbose=0)
        labels = np.concatenate([y.numpy() for _, y in dataset])

        sil_score = silhouette_score(embeddings, labels) # mide separabilidad entre clusters, es decir que tan cercanos suelen estar entre los mismos estilos

        nn = NearestNeighbors(n_neighbors=6, metric='cosine').fit(embeddings) # evalúa si los 5 vecinos más cercanos poseen misma etiqueta
        distances, indices = nn.kneighbors(embeddings)

        correct = 0
        for i, neighbors in enumerate(indices):
            true_label = labels[i]
            retrieved_labels = labels[neighbors[1:]]
            correct += np.sum(retrieved_labels == true_label)
        precision_at_5 = correct / (len(labels) * 5)

        if not silent:
            print(f"[Eval] Silhouette Score: {sil_score:.4f} | Precision@5: {precision_at_5:.4f}")
        else:
            print(f"[Epoch Eval] Silhouette={sil_score:.4f}, P@5={precision_at_5:.4f}")

    except Exception as e:
        print(f"[Eval] Error: {str(e)}")

## Resultados

### Accuracy
![EfficientNet-B2](https://i.imgur.com/jJBtLli.png)

* **Fase 1** muestra una rápida mejora tanto en entrenamiento como validación, alcanzando un aproximado 50% en validación.
* **Fase 2**  mejora un poco más y supera ligeramente los resultados de la Fase 1
* **Fase 3**, aunque mejora progresivamente, queda por debajo de las fases anteriores en precisión, especialmente en validación.

---



### Loss
![EfficientNet-B2](https://i.imgur.com/stwnHHS.png)

* **Fase 1** comienza con pérdida más baja en validación (\1.6) y se estabiliza rápidamente.
* **Fase 2** muestra una gran caída de pérdida en entrenamiento, aún asi la pérdida de validación se mantiene creciente, posible sobreajuste
* **Fase 3** muestra pérdidas más altas, además  de una brecha constante entre entrenamiento y validación

---



### Precision@5
![EfficientNet-B2](https://i.imgur.com/naOr8lf.png)

* **Fase 1** el mejor de todas las fases, superando 0.43 en la última época.
* **Fase 2** al inicio mejorapero no alcanza a la fase 1
* **Fase 3** tiene el peor desempeño de las fases, inmejorable.

Es decir los embeddings de la fase 1 son los más útiles para recuperación por similitud, y el ajuste fino aplicado luego pareciese desestabilizar las representaciones, haciendolo inutiles

---



### Silhouette Score
![EfficientNet-B2](https://i.imgur.com/YVCxris.png)

* Todos los valores son negativos, dado que las clases pueden estar solapadas o la representación aún no es muy discriminativa
* **Fase 1** muestra mejoras suaves y es la única en llegar a un valor cercano a cero (+0.0006).
* **Fase 2 y 3** empeoran consistentemente, lo que hace a los embeddings menos distinguibles

Degradacion de calidad o representatividad de embeddings

